# How to get log probabilities

:::info Prerequisites

This guide assumes familiarity with the following concepts:
- [Chat models](/docs/concepts/#chat-models)

:::

Certain chat models can be configured to return token-level log probabilities representing the likelihood of a given token. This guide walks through how to get this information in LangChain.

## OpenAI

Install the LangChain x OpenAI package and set your API key

In [1]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

StdinNotImplementedError: getpass was called, but this frontend does not support input requests.

For the OpenAI API to return log probabilities we need to configure the `logprobs=True` param. Then, the logprobs are included on each output [`AIMessage`](https://python.langchain.com/v0.2/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html) as part of the `response_metadata`:

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini").bind(logprobs=True)

msg = llm.invoke(("human", "how are you today"))

msg.response_metadata["logprobs"]["content"][:5]

[{'token': "I'm",
  'bytes': [73, 39, 109],
  'logprob': -0.06202448,
  'top_logprobs': []},
 {'token': ' just',
  'bytes': [32, 106, 117, 115, 116],
  'logprob': -0.040247045,
  'top_logprobs': []},
 {'token': ' a',
  'bytes': [32, 97],
  'logprob': -9.233804e-05,
  'top_logprobs': []},
 {'token': ' computer',
  'bytes': [32, 99, 111, 109, 112, 117, 116, 101, 114],
  'logprob': -0.3915352,
  'top_logprobs': []},
 {'token': ' program',
  'bytes': [32, 112, 114, 111, 103, 114, 97, 109],
  'logprob': -5.669615e-05,
  'top_logprobs': []}]

And are part of streamed Message chunks as well:

In [4]:
ct = 0
full = None
for chunk in llm.stream(("human", "how are you today")):
    if ct < 5:
        full = chunk if full is None else full + chunk
        if "logprobs" in full.response_metadata:
            print(full.response_metadata["logprobs"]["content"])
    else:
        break
    ct += 1

[]
[{'token': "I'm", 'bytes': [73, 39, 109], 'logprob': -0.06202471, 'top_logprobs': []}]
[{'token': "I'm", 'bytes': [73, 39, 109], 'logprob': -0.06202471, 'top_logprobs': []}, {'token': ' just', 'bytes': [32, 106, 117, 115, 116], 'logprob': -0.031950008, 'top_logprobs': []}]
[{'token': "I'm", 'bytes': [73, 39, 109], 'logprob': -0.06202471, 'top_logprobs': []}, {'token': ' just', 'bytes': [32, 106, 117, 115, 116], 'logprob': -0.031950008, 'top_logprobs': []}, {'token': ' a', 'bytes': [32, 97], 'logprob': -6.2775536e-05, 'top_logprobs': []}]
[{'token': "I'm", 'bytes': [73, 39, 109], 'logprob': -0.06202471, 'top_logprobs': []}, {'token': ' just', 'bytes': [32, 106, 117, 115, 116], 'logprob': -0.031950008, 'top_logprobs': []}, {'token': ' a', 'bytes': [32, 97], 'logprob': -6.2775536e-05, 'top_logprobs': []}, {'token': ' computer', 'bytes': [32, 99, 111, 109, 112, 117, 116, 101, 114], 'logprob': -0.39101082, 'top_logprobs': []}]


## Next steps

You've now learned how to get logprobs from OpenAI models in LangChain.

Next, check out the other how-to guides chat models in this section, like [how to get a model to return structured output](/docs/how_to/structured_output) or [how to track token usage](/docs/how_to/chat_token_usage_tracking).